# Spam Filtering Using Naive Bays

In [3]:
import glob
import pandas as pd
import os

In [4]:
print (os.getcwd()) # this helps to see currently which location the control exist

C:\Users\inkpathak\Desktop\Anuj Gupta\enron1


In [6]:
os.chdir('C:\\Users\\inkpathak\\Desktop\\Anuj Gupta\\enron1') # Provide the path here where all the folders of spam and ham mails are located

Generally you may receive multiple text files on Enron dataset which needs to be merged and leveled for further analysis. We are going to create a data frame file containing two columns 
        Text -> This column would contain the text body of each mail
        Class -> This column contains the lebel of whether the mail is 'spam' or 'ham'

In [7]:
spam_data = pd.DataFrame(columns = ("Text", "Class")) 
# creating a data-frame with two columns Text= the extracted texts from each mail and Class represents whether it is "Spam" and "Ham"

In [8]:
for cla in glob.glob("dataset/*"): # Here the folder name is data set in which there are two sub-folder "Spam" & "Ham"
    clas = cla.split(os.sep)[1]    # We are splitting the folder names as class using OS-Seperator and taking the 2nd item in the list
    for file in glob.glob(cla + "/*.txt"): # Here we are deep diving in each of the folder and reading the text files one by one
        text = open(file, "r", encoding = "ISO-8859-1").read() # Reading the file , for Windows generally we need to mention the encoding 
        text = " ".join(text.split("\n")) # Splitting the text files and rejoining into a single text
        spam_data = spam_data.append(pd.Series([text, clas], index = ["Text", "Class"]), ignore_index = True) # continious append to the data frame

In [ ]:
spam_data.to_csv('./datasets/email_classification.csv',index=False,header=True) # Writing the CSV file in to the location mentioned

In [ ]:
# reading data 
Location = r'C:\Users\inkpathak\Desktop\Anuj Gupta\enron1\email_classification.csv'
df = pd.read_csv(Location)

In [46]:
spam_data.head()

,Text,Class
0,Subject: christmas tree farm pictures,0
1,"Subject: vastar resources , inc . gary , produ...",0
2,Subject: calpine daily gas nomination - calpin...,0
3,Subject: re : issue fyi - see note below - alr...,0
4,Subject: meter 7268 nov allocation fyi . - - -...,0


In [47]:
spam_data.shape

(5172, 2)

In [13]:

from collections import Counter
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix 
# Create a dictionary of words with its frequency
import os
import numpy as np 
import scipy as sp 
import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 
import pandas as pd 
import nltk
import re
import csv
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
from scipy.sparse import hstack
from sklearn.metrics import confusion_matrix
from time import time

C:\Users\inkpathak\AppData\Local\Continuum\anaconda2\envs\py34t\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
# convert label to a numerical variable
spam_data['Class'] = spam_data.Class.map({'ham':0, 'spam':1})

In [14]:
stop = set(stopwords.words('english'))

In [15]:
spam_complete = spam_data['Text'].tolist()

In [17]:
len(spam_complete)

5172

In [38]:
spam_complete[1]

'Subject: vastar resources , inc . gary , production from the high island larger block a - 1 # 2 commenced on saturday at 2 : 00 p . m . at about 6 , 500 gross . carlos expects between 9 , 500 and 10 , 000 gross for tomorrow . vastar owns 68 % of the gross production . george x 3 - 6992 - - - - - - - - - - - - - - - - - - - - - - forwarded by george weissman / hou / ect on 12 / 13 / 99 10 : 16 am - - - - - - - - - - - - - - - - - - - - - - - - - - - daren j farmer 12 / 10 / 99 10 : 38 am to : carlos j rodriguez / hou / ect @ ect cc : george weissman / hou / ect @ ect , melissa graves / hou / ect @ ect subject : vastar resources , inc . carlos , please call linda and get everything set up . i \' m going to estimate 4 , 500 coming up tomorrow , with a 2 , 000 increase each following day based on my conversations with bill fischer at bmar . d . - - - - - - - - - - - - - - - - - - - - - - forwarded by daren j farmer / hou / ect on 12 / 10 / 99 10 : 34 am - - - - - - - - - - - - - - - - - -

In [43]:
def clean(doc):
    doc = " ".join([i.replace('*', '') for i in doc.lower().split()])
    doc = " ".join([i.replace(':', ' ') for i in doc.split()])
    doc = " ".join([i.replace('.', ' ') for i in doc.split()])
    doc = " ".join([i.replace('=', '') for i in doc.split()])
    doc = " ".join([i.replace('/', ' ') for i in doc.split()])
    doc = " ".join([i.replace(')', ' ') for i in doc.split()])
    doc = " ".join([i.replace('(', ' ') for i in doc.split()])
    doc = " ".join([i.replace('"', ' ') for i in doc.split()])
    doc = " ".join([i.replace('-', ' ') for i in doc.split()])
    doc = " ".join([i.replace('_', ' ') for i in doc.split()])
    doc = " ".join([i for i in doc.split() if not i.isdigit()])
    doc = " ".join([i for i in doc.split() if i.isalpha()])
    doc = " ".join([i for i in doc.split() if i not in stop])
    return doc

In [44]:
spam_clear = [clean(doc) for doc in spam_complete]

In [50]:
spam_clear[7]

'subject duns number changes fyi forwarded gary l payne hou ect pm antoine v pierre pm tommy j yanowski hou ect ect kathryn bussell hou ect ect gary l payne hou ect ect diane e niestrath hou ect ect romeo souza hou ect ect michael eiben hou ect ect clem cernosek hou ect ect scotty gilbert hou ect ect dave nommensen hou ect ect david rohan hou ect ect kevin heal cal ect ect richard pinion hou ect ect cc mary g gosnell hou ect ect jason moore hou ect ect samuel schott hou ect ect bernice rodriguez hou ect ect subject duns number changes making changes wednesday december agree problem dnb number change please notify otherwise make change scheduled dunns number change counterparty cp id number cinergy resources inc energy dynamics management inc south jersey resources group llc transalta energy marketing us inc philadelphia gas works thanks rennie'

In [ ]:
dictionary = corpora.Dictionary(doc_clean)

In [51]:
spam_data["clean_text"] = spam_clear

In [52]:
spam_data.head()

,Text,Class,clean_text
0,Subject: christmas tree farm pictures,0,subject christmas tree farm pictures
1,"Subject: vastar resources , inc . gary , produ...",0,subject vastar resources inc gary production h...
2,Subject: calpine daily gas nomination - calpin...,0,subject calpine daily gas nomination calpine d...
3,Subject: re : issue fyi - see note below - alr...,0,subject issue fyi see note already done stella...
4,Subject: meter 7268 nov allocation fyi . - - -...,0,subject meter nov allocation fyi forwarded lau...


In [53]:
spam_data_ABT=spam_data[["Class","clean_text"]]

In [54]:
spam_data_ABT.head()

,Class,clean_text
0,0,subject christmas tree farm pictures
1,0,subject vastar resources inc gary production h...
2,0,subject calpine daily gas nomination calpine d...
3,0,subject issue fyi see note already done stella...
4,0,subject meter nov allocation fyi forwarded lau...


In [55]:
# how to define X and y (from the mail spam data) for use with COUNTVECTORIZER
X = spam_data_ABT.clean_text
y = spam_data_ABT.Class
print(X.shape)
print(y.shape)

(5172,)
(5172,)


In [56]:
# split X and y into training and testing sets
# by default, it splits 75% training and 25% test
# random_state=1 for reproducibility
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3879,)
(1293,)
(3879,)
(1293,)


#### Vectorizing our dataset

In [57]:
# 2. instantiate the vectorizer
vect = CountVectorizer()

In [58]:
# learn training data vocabulary, then use it to create a document-term matrix

# 3. fit
vect.fit(X_train)

# 4. transform training data
X_train_dtm = vect.transform(X_train)

In [59]:
# equivalently: combine fit and transform into a single step
# this is faster and what most people would do
X_train_dtm = vect.fit_transform(X_train)

In [60]:
# examine the document-term matrix
X_train_dtm

<3879x39159 sparse matrix of type '<class 'numpy.int64'>'
	with 242859 stored elements in Compressed Sparse Row format>

In [61]:
# 4. transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

# you can see that the number of columns, 7456, is the same as what we have learned above in X_train_dtm

<1293x39159 sparse matrix of type '<class 'numpy.int64'>'
	with 70412 stored elements in Compressed Sparse Row format>

In [62]:
# 1. import
from sklearn.naive_bayes import MultinomialNB

# 2. instantiate a Multinomial Naive Bayes model
nb = MultinomialNB()

In [63]:
# 3. train the model 
# using X_train_dtm (timing it with an IPython "magic command")

%time nb.fit(X_train_dtm, y_train)

Wall time: 16 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [64]:
# 4. make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [65]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.9775715390564579

In [66]:
# examine class distribution
print(y_test.value_counts())
# there is a majority class of 0 here, hence the classes are skewed

# calculate null accuracy (for multi-class classification problems)
# .head(1) assesses the value 1208
null_accuracy = y_test.value_counts().head(1) / len(y_test)
print('Null accuracy:', null_accuracy)

# Manual calculation of null accuracy by always predicting the majority class
print('Manual null accuracy:',(1208 / (1208 + 185)))

0    921
1    372
Name: Class, dtype: int64
Null accuracy: 0    0.712297
Name: Class, dtype: float64
Manual null accuracy: 0.8671931083991385


In [67]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[911,  10],
       [ 19, 353]])

In [68]:
# print message text for the false positives (ham incorrectly classified as spam)

X_test[y_pred_class > y_test]

# alternative less elegant but easier to understand
# X_test[(y_pred_class==1) & (y_test==0)]

3597    subject important video announcement important...
3539    subject dear wish find mission kosovo find new...
2456    subject hunter singing christmas program video...
126     subject fw quips remember amateurs built ark p...
2068    subject skydive spaceland specials skydive spa...
301     subject expatriate zone issue expatriate zone ...
2772    subject boat believe boat ft long boat cover b...
2479    subject weekly fan fares delta fan fares febru...
304     subject http www pge texas com www gtt nsf htm...
107     subject new email yo bro new email address sav...
Name: clean_text, dtype: object

In [69]:
# print message text for the false negatives (spam incorrectly classified as ham)

X_test[y_pred_class < y_test]
# alternative less elegant but easier to understand
# X_test[(y_pred_class=0) & (y_test=1)]

3835                 subject multiple ways get home loans
3789                                              subject
4932    subject hello mail transaction failed partial ...
3743                              subject holiday e cards
4728                                              subject
4423                                        subject enjoy
5168          subject str rndlen extra time word bodyhtml
4646    subject investigation tue jan sir trouble gett...
4657                               subject hi try nothing
3971    subject thu may first gove rnment mo rtgage pr...
3830           subject document please read attached file
3815    subject feeling slze johnson rtxyj nxfgr ktrqr...
4181    subject network wed aug kttihkefi iewmeh ckust...
4684                                              subject
4830                                         subject note
4498    subject promised help people please refer ques...
3748                    subject holiday e cards gbhzivjwl
4593          

In [70]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)

# Numpy Array with 2C
# left Column: probability class 0
# right C: probability class 1
# we only need the right column 
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

# Naive Bayes predicts very extreme probabilites, you should not take them at face value

array([1.08465903e-022, 8.98003374e-030, 2.86781696e-043, ...,
       2.19034125e-009, 7.51472506e-027, 1.85378583e-113])

In [71]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.9966405146346304

In [72]:
# 1. import
from sklearn.linear_model import LogisticRegression

# 2. instantiate a logistic regression model
logreg = LogisticRegression()

In [73]:
# 3. train the model using X_train_dtm
%time logreg.fit(X_train_dtm, y_train)

Wall time: 575 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [74]:
# 4. make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [75]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([5.07731912e-05, 3.21678236e-03, 1.67163834e-05, ...,
       2.88866399e-03, 1.10065759e-04, 1.19296726e-12])

In [76]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

0.9791183294663574

In [77]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

0.9967397522561965

Examining a model for further insight

In [78]:
# store the vocabulary of X_train
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)

39159

In [79]:
# rows represent classes, columns represent tokens
nb.feature_count_.shape


(2, 39159)

In [80]:
# number of times each token appears across all HAM messages
ham_token_count = nb.feature_count_[0, :]
ham_token_count

array([1., 0., 0., ..., 1., 0., 0.])

In [81]:
# number of times each token appears across all SPAM messages
spam_token_count = nb.feature_count_[1, :]
spam_token_count

array([15.,  1.,  1., ...,  0.,  1.,  1.])

In [82]:
# create a DataFrame of tokens with their separate ham and spam counts
tokens = pd.DataFrame({'token':X_train_tokens, 'ham':ham_token_count, 'spam':spam_token_count}).set_index('token')
tokens.head()

,ham,spam
token,,
aa,1.0,15.0
aaa,0.0,1.0
aabvmmq,0.0,1.0
aac,0.0,1.0
aachecar,1.0,0.0


In [83]:
# examine 5 random DataFrame rows
# random_state=6 is a seed for reproducibility
tokens.sample(5, random_state=6)

,ham,spam
token,,
additive,0.0,1.0
baseboard,0.0,2.0
frazier,0.0,2.0
mesenteric,0.0,1.0
ferc,38.0,0.0


In [85]:
# Naive Bayes counts the number of observations in each class
nb.class_count_

array([2751., 1128.])